In [ ]:
# Install required libraries
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
!pip install -qU "langchain-nvidia-ai-endpoints"
!pip install -qU langchain-astradb
!pip install PyPDF2 pypdf

In [ ]:
# Get API keys and endpoints from Colab userdata
from google.colab import userdata
key = userdata.get('llm_projects')
db_api_endpoint = userdata.get('ASTRA_DB_API_ENDPOINT')
db_application_token = userdata.get('ASTRA_DB_APPLICATION_TOKEN')
langsmith_api_key = userdata.get('LANGSMITH_API_KEY')

import os
os.environ["NVIDIA_API_TOKEN"] = key
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key

In [ ]:
# Import and initialize ChatNVIDIA model
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

client = ChatNVIDIA(
  model="meta/llama3-70b-instruct",
  api_key=key,
  temperature=0.5,
  top_p=1,
  max_tokens=1024,
)

#checking....
for chunk in client.stream([{"role":"user","content":"what is an AI agent?"}]):
  print(chunk.content, end="")

#Initialize NVIDIA embeddings
embeddings = NVIDIAEmbeddings(model="NV-Embed-QA", api_key=key)

In [ ]:
# Import necessary libraries for RAG
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore

# Load PDF
loader = PyPDFLoader("22365_19_Agents_v8.pdf")
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
doc_chunks = text_splitter.split_documents(docs)

# Initialize AstraDB Vector Store
vector_store = AstraDBVectorStore(
    embedding=embeddings,
    api_endpoint=db_api_endpoint,
    collection_name="rag_db",
    token=db_application_token,
)

# Add document chunks to the vector store
_ = vector_store.add_documents(documents=doc_chunks)

In [ ]:
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

# Pull RAG prompt from the hub
prompt = hub.pull("rlm/rag-prompt")

# Define state for the RAG application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define retrieve step for the RAG application
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

# Define generate step for the RAG application
def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = client.invoke(messages)
    return {"answer": response.content}


# Compile the RAG application graph
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Invoke the RAG graph with a question and print the response
response = graph.invoke({"question": "What is an agent?"})
print(response["answer"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


An agent is an application that attempts to achieve a goal by observing the world and acting upon it using the tools at its disposal. Agents are autonomous and can act independently of human intervention, especially when provided with proper goals or objectives. They can also be proactive in their approach to reaching their goals.
